02.03.2020
###  Работа с данными 
## Семинар 1

***

**Задача 1**

2/3 трафика идет на прямые ссылки из поисковых систем, конверсия в просмотр 10%. Остальной трафик идет на ЦПБ, просмотров из него 30%. Найти общую конверию в просмотры.

Решение:

Событие $А$ - просмотр,
$H_1$ - прямой переход, $P(H_1) = 2/3$, $P(A|H_1) = 0.1$

$H_2$ - переход из ЦПБ, $P(H_2) = 1/3$, $P(A|H_2) = 0.3$

По формуле полной вероятности $P(A)= P(H_1)P(A|H_1)+P(H_2)P(A|H_2)$

In [11]:
pA = 2/3*0.1+1/3*0.3
print(f"Конверсия по обоим каналам равна {pA*100:.2f}%")

Конверсия по обоим каналам равна 16.67%


***

**Задача 2. Титаник** 

По данным колонок Class и Survived из датасета пассажиров Титаника найти вероятность выжить для пассажира 1 класса

$$
P(Survived = Yes | Class= 1st) = \frac{P(Class=1 | Survived = Yes) \cdot P(Survived = Yes)}{P(Class=1)}
$$

Решение:

В задачах с реальными данными вероятности заменяются частотами:

In [66]:
#загрузка датасета
import pandas as pd

df = pd.read_csv('Dataset.data', sep = ',', header=None, 
                 names=['Class','Age','Gender','Survived'], usecols = ['Class', 'Survived'])
df.head()

,Class,Survived
0,1st,yes
1,1st,yes
2,1st,yes
3,1st,yes
4,1st,yes


In [71]:
total_num = df.count()[0] #2201
pS = df[df.Survived == 'yes'].count()[0]/total_num #711

p1 = df[df.Class == '1st'].count()[0]/total_num #325

p1S = df[(df.Class == '1st') & (df.Survived=='yes')].count()[0]/df[df.Survived == 'yes'].count()[0] #203/711

pS1 = p1S*pS/p1
print(f"Вероятность выжить, если пассажир путешествовал первым классом {pS1*100:.2f} %")
print(f"Априорная вероятность выжить  {pS*100:.2f} %")

Вероятность выжить, если пассажир путешествовал первым классом 62.46 %
Априорная вероятность выжить  32.30 %


In [72]:
# а вообще можно было сделать сразу так

pS1a = df[(df.Class == '1st') & (df.Survived=='yes')].count()[0]/df[df.Class == '1st'].count()[0]
print(f"Вероятность выжить, если пассажир путешествовал первым классом {pS1a*100:.2f} %")

Вероятность выжить, если пассажир путешествовал первым классом 62.46 %


***

**Задача 3. Байесовский классификатор текстов**



С помощью теоремы Байеса определить по тексту сообщения его класс (spam/ham).

**Обобщение предыдущей задачи на многомерный случай:**

$$
c_{MAP} = \arg \max_{\substack{c \in C}}P(c \mid X) = \arg \max_{\substack{c \in C}}\frac{P(c)P(X\mid c)}{P(X)}
$$

$$
c_{MAP} = \arg \max_{\substack{c \in C}}P(c)P(x_1\ldots x_n\mid c)
$$

$P(c)$ вычисляем как # сообщений класса/общее # сообщений.

$P(x_1\ldots x_n\mid c)$ в предположении о независимочти $x_i$ вычисляется как
$$
P(x_1\ldots x_n\mid c) = P(x_1 \mid c)\cdot \ldots \cdot P(x_n\mid c) = \prod_{i=1}^nP(x_i\mid c)
$$

Обучение наивного байесовского классификатора сводиться к вычислению по корпусу текстов (тренировочных данных) относительных частот по категориям, тогда мы получаем т.н *multinomial bayes model*:
$$
\forall i,j: P(x_i \mid c_j) = \frac{n_{c_j}(x_i)}{\sum_{k\in V}n_{c_k}(x_i)}
$$
где $n_{c_j}(x_i)$ - количество раз, которое слово $x_i$ встречается в теме $c_j$, а $V$ - *словарь* корпуса документов, множество всех уникальных слов

**Тупой способ: без оформления в функцию**

In [75]:
pwd

'C:\\Users\\79850\\Documents\\MAI-data'

In [8]:
import pandas as pd

filename = '..\\SMSSpamCollection'
df = pd.read_csv(
    filename,
    sep='\t',
    header=None,
    names=['class','sms'])
df.head()

,class,sms
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [2]:
num_objects, num_features = df.shape

In [3]:
SPAM_CLASS = 'spam'
HAM_CLASS = 'ham'

In [4]:
spam_sms_num = (df['class'] == SPAM_CLASS).sum()
ham_sms_num = (df['class'] == HAM_CLASS).sum()


# априорные вероятности 
p_spam = spam_sms_num / num_objects
p_ham = ham_sms_num / num_objects

In [5]:
print(f'{p_spam:.4f}, {p_ham:.4f}')

0.1341, 0.8659


In [9]:
test_word = 'Free'.lower()

In [10]:
# предварительная обработка строки
import string

sms_example = df['sms'].values[5] # одна строка для примера
sms_example = ''.join([char for char in sms_example if char not in string.punctuation]) # удаляем знаки препинания
sms_example = ' '.join([word.lower() for word in sms_example.split()]) # приводим слова к нижнему регистру
sms_example

'freemsg hey there darling its been 3 weeks now and no word back id like some fun you up for it still tb ok xxx std chgs to send £150 to rcv'

In [11]:
# оформим это как функцию

def text_preprocess(sms_text: str):
    """Преобразоавние текста для анализа"""
    text_no_punctuation = ''.join([char for char in sms_text if char not in string.punctuation])
    text_lowercase = ' '.join([word.lower() for word in text_no_punctuation.split()])
     
    return text_lowercase

In [14]:
df = df.assign(processed_text = df['sms'].apply(text_preprocess)) # добавляем колонку с обработанным текстом
df.head()

,class,sms,processed_text
0,ham,"Go until jurong point, crazy.. Available only ...",go until jurong point crazy available only in ...
1,ham,Ok lar... Joking wif u oni...,ok lar joking wif u oni
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,free entry in 2 a wkly comp to win fa cup fina...
3,ham,U dun say so early hor... U c already then say...,u dun say so early hor u c already then say
4,ham,"Nah I don't think he goes to usf, he lives aro...",nah i dont think he goes to usf he lives aroun...


In [16]:
#вероятность встретить тестовое слово в спаме

spam_test_word_entries = df[df['class']==SPAM_CLASS]['processed_text'].apply(lambda row: test_word in row).sum()

In [17]:
spam_test_word_entries/ spam_sms_num

0.26639892904953144